In [22]:
#####
# Packages
##### 

import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras

# Clean Data Section (from preparedata.py)

In [2]:
#####
# Packages
##### 

import numpy as np
import pandas as pd 
import re #for regex search purposes          
import pickle
cleanfilename = 'cleandata.pkl'
wikihowfile = 'wikihowSep.csv'


# In[19]:


sepdata = pd.read_csv(wikihowfile)


# In[20]:


print ('Read file. Shape is ', sepdata.shape)


# In[21]:


sepdata_v1 = sepdata.dropna(subset=['headline','text'], axis=0).reset_index(drop=True) 


# In[22]:


print ('Removed NAs. Shape is ', sepdata_v1.shape)


# In[23]:


# List of contractions that we will map to 

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}


# In[24]:


def text_cleaner(text):
    # Step 0: Convert to string in case a float or int is found.
    newString = str(text)
    # Step 1: Lower case the text 
    newString = newString.lower()
    # Step 2: Get rid of commas
    #newString = re.sub(r'\([^)]*\)', '', newString)
    # Step 3: Get rid of quotations 
    newString = re.sub('"','', newString)
    # Step 4: get rid of contractions with our contraction mapping 
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])  
    # Step 5: get rid of the \n stuff 
    newString = re.sub(r"'s\n","",newString)
    # Step 6: get rid of most special characters
    newString = re.sub(r'[?$()_\-—\d{}/#&%<>=@\*~:;\\\+\']',r' ', newString) 
    # Step 7: separating punctuations so they are their own tokens
    newString = re.sub(r'([,.!;:?])', r' \1 ', newString)

    return newString.split()


# In[25]:


print("Starting to clean data")


# In[26]:


clean_data = pd.DataFrame()

clean_data['text'] = sepdata_v1['text'].apply(text_cleaner)


# In[27]:


print("Text clean complete. Number of items is ", len(clean_data.text))


# In[28]:


clean_data['headline'] = sepdata_v1['headline'].apply(text_cleaner)


# In[29]:


print("Title cleaning complete. Number of items is ", len(clean_data.headline))


# In[30]:


print('Saving file to disk. File name is ', cleanfilename)


# In[31]:


pickle.dump( clean_data, open( cleanfilename, "wb" ) )


Read file. Shape is  (1585695, 5)
Removed NAs. Shape is  (1387290, 5)
Starting to clean data
Text clean complete. Number of items is  1387290
Title cleaning complete. Number of items is  1387290
Saving file to disk. File name is  cleandata.pkl


In [6]:
!pwd

/Users/vlu/Desktop


In [8]:
embeddings_index = {}
#f = open('data/glove/glove.6B/glove.6B.300d.txt')
f = open('/Users/vlu/Downloads/glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [9]:
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [12]:
from nltk.corpus import stopwords #stopwords that are provided to us via nltk 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import math

In [13]:
def doc2seq(texts, MAX_NB_WORDS, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    index_word = tokenizer.index_word
    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
    
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return data, embedding_matrix, word_index, index_word

In [14]:
MAX_NB_WORDS = 30000
MAX_TEXT_LENGTH = 100
EMBEDDING_DIM = 300
data = clean_data.text

x_data, encoder_emb, x_word_index, x_index_word = doc2seq(data, MAX_NB_WORDS, MAX_TEXT_LENGTH, EMBEDDING_DIM)

Found 212813 unique tokens.


In [15]:
MAX_NB_WORDS = 30000
MAX_HEADLINE_LENGTH = 30
EMBEDDING_DIM = 300
data = clean_data.headline

y_data, decoder_emb, y_word_index, y_index_word = doc2seq(data, MAX_NB_WORDS, MAX_HEADLINE_LENGTH, EMBEDDING_DIM)

Found 78682 unique tokens.


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train, x_test_temp, y_train, y_test_temp = train_test_split(x_data, y_data, 
                                                            test_size=0.3, random_state=0) 

In [18]:
x_dev, x_test, y_dev, y_test = train_test_split(x_test_temp, y_test_temp, 
                                                            test_size=0.33, random_state=0) 

In [105]:
from tensorflow.keras.layers import Dense,LSTM,Dropout,Input,Activation,Add,concatenate, Embedding, RepeatVector
from tensorflow.keras.models import Model
#from tensorflow.keras.callbacks import EarlyStopping
from attention_keras.layers.attention import AttentionLayer

from tensorflow.keras import backend as K 
K.clear_session() 
hidden_units = 250 #this is 600 in the paper. To be changed later.

In [106]:
enc_embedding_layer = Embedding(len(x_word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[encoder_emb],
                            input_length=MAX_TEXT_LENGTH,
                            trainable=False,
                            name='EncoderEmbeddingLayer')

In [107]:
dec_embedding_layer = Embedding(len(y_word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[decoder_emb],
                            input_length=MAX_HEADLINE_LENGTH,
                            trainable=False,
                            name='DecoderEmbeddingLayer')

In [109]:
# https://github.com/DeepsMoseli/Bidirectiona-LSTM-for-text-summarization-/blob/master/lstm_Attention.py
#https://github.com/rstudio/keras/issues/624

# Encoder
encoder_inputs = Input(shape=(MAX_TEXT_LENGTH,), name='EncoderInput')
encoder_embedding = enc_embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(hidden_units, return_state=True)
encoder_LSTM_R = LSTM(hidden_units, return_state=True, go_backwards=True)
encoder_outputs_R, state_h_R, state_c_R = encoder_LSTM_R(encoder_embedding)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

final_h = Add()([state_h, state_h_R])
final_c = Add()([state_c, state_c_R])
encoder_states = [final_h, final_c]

#decoder
decoder_inputs = Input(shape=(None,), name='DecoderInput')
# decoder_inputs = Input(shape=(None,), name = 'DecoderInput') 
decoder_embedding = dec_embedding_layer(decoder_inputs)
decoder_LSTM = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=encoder_states) 

#decoder_dense = Dense(len(y_word_index)+1, activation='softmax') #this one owrks with adam
#decoder_outputs = decoder_dense(decoder_outputs) #This one works  with adam
# LAST TWO: 
decoder_dense = TimeDistributed(Dense(len(y_word_index)+1, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_outputs) 

In [102]:
# modeling
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
#model.compile(optimizer='adam', loss="mse", metrics=["accuracy"])
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy') #switch this to adam if needed

In [103]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
EncoderInput (InputLayer)       [(None, 100)]        0                                            
__________________________________________________________________________________________________
EncoderEmbeddingLayer (Embeddin (None, 100, 300)     63844200    EncoderInput[0][0]               
__________________________________________________________________________________________________
DecoderInput (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 250), (None, 551000      EncoderEmbeddingLayer[0][0]      
______________________________________________________________________________________________

In [104]:
history=model.fit([x_train,np.hstack((np.zeros((y_train.shape[0],1)), y_train[:, :-1]))], 
                  y_train,
                  epochs=1,
                  batch_size=160, 
                  validation_data=([x_dev,np.hstack((np.zeros((y_dev.shape[0],1)), y_dev[:, :-1]))], y_dev)
                 )

Train on 971103 samples, validate on 278845 samples
   160/971103 [..............................] - ETA: 35:21:46 - loss: 11.2732

KeyboardInterrupt: 